In [ ]:
# Librerias
!pip install -q transformers datasets sentence-transformers evaluate scipy scikit-learn accelerate seaborn matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

print("Descargando dataset STS Benchmark...")
dataset = load_dataset("mteb/stsbenchmark-sts")

df_train = pd.DataFrame(dataset['train'])
df_dev = pd.DataFrame(dataset['validation'])
df_test = pd.DataFrame(dataset['test'])

print("\n--- ESTADÍSTICAS DEL DATASET ---")
print(f"Cantidad de ejemplos en Train: {len(df_train)}")
print(f"Cantidad de ejemplos en Validation: {len(df_dev)}")
print(f"Cantidad de ejemplos en Test: {len(df_test)}")

print("\n--- EJEMPLO DE DATOS ---")
print(df_train[['sentence1', 'sentence2', 'score']].head())
plt.figure(figsize=(8, 5))
sns.histplot(df_train['score'], bins=20, kde=True, color='blue')
plt.title("Distribución de las puntuaciones de similitud (Conjunto de Train)")
plt.xlabel("Puntuación de Similitud (0.0 a 5.0)")
plt.ylabel("Frecuencia")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

OBJETIVO Z1
Implementación del código para N-Gramas (contar palabras coincidentes) y Sentence Embeddings (usar IA pre-enternada).

**IMPORTANTE**: Ngramas compara letras (mismas palabras - no sinonimos), mientras que embedding compara las ideas (significado de las palabras)

In [ ]:
# BLOQUE: OBJETIVO Z1 (MODELOS NO SUPERVISADOS)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from scipy.stats import pearsonr

# Función auxiliar para evaluar (Calcula Pearson)
def evaluate_pearson(predictions, references):
    pearson_corr, _ = pearsonr(predictions, references)
    return pearson_corr

test_sentences1 = df_test['sentence1'].tolist()
test_sentences2 = df_test['sentence2'].tolist()
gold_scores = df_test['score'].tolist()

# --- MODELO A: N-GRAMAS (Baseline básica) ---
print("1. Ejecutando modelo de N-Gramas...")

vectorizer = CountVectorizer().fit(test_sentences1 + test_sentences2)
vecs1 = vectorizer.transform(test_sentences1)
vecs2 = vectorizer.transform(test_sentences2)

ngram_preds = []
for i in range(len(test_sentences1)):
    sim = cosine_similarity(vecs1[i], vecs2[i])[0][0]
    ngram_preds.append(sim * 5)

# --- MODELO B: SENTENCE EMBEDDINGS (Baseline avanzada) ---
print("2. Ejecutando modelo de Sentence Embeddings (all-MiniLM-L6-v2)")

model_st = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings1 = model_st.encode(test_sentences1, convert_to_tensor=True)
embeddings2 = model_st.encode(test_sentences2, convert_to_tensor=True)


from sentence_transformers import util
cosine_scores = util.cos_sim(embeddings1, embeddings2)

st_preds = []
for i in range(len(test_sentences1)):
    st_preds.append(cosine_scores[i][i].item() * 5)

# EVALUACIÓN FINAL Z1
print("\n RESULTADOS Z1 (Correlación de Pearson)")
#LES PASAMOS LAS NOTAS PREDICHAS PARA NGRAMS Y PARA ST Y LOS RESULTADOS REALES
pearson_ngram = evaluate_pearson(ngram_preds, gold_scores)
pearson_st = evaluate_pearson(st_preds, gold_scores)

print(f"Modelo N-Gramas: {pearson_ngram:.4f}")
print(f"Modelo Sentence Embeddings: {pearson_st:.4f}")

results_table = {
    "Modelo": ["N-Grams (Z1)", "Sentence Embeddings (Z1)"],
    "Pearson": [pearson_ngram, pearson_st]
}
pd.DataFrame(results_table)

**Z2: AJUSTAR EL MODELO A LA TAREA UTILIZANDO LOS DATOS DE ENTRENAMIENTO (FINE-TUNNING)**

A utilizar: Cross-Encoder, como entrada se le dan dos frases juntas separadas por una etiqueta [SEP]. Después, como cerebro se usará BERT con su mecanismo de auto atención para mirar cada palabra de A y compararla con cada palabra de B. Y, finalmente, le pediremos que escupa un solo número (regresión) que represente la similitud.

BLOQUE 2.1: PREPARAR LOS DATOS PARA EL EXAMEN
Vaciamos la memorioa de la tarjeta grafica y cargamos el dataset de 0 para evitar columnas fantasma que puedan darnos problemas.

In [ ]:
# --- BLOQUE Z2 - PASO 1: LIMPIEZA Y CARGA ---
import torch
import gc
from datasets import load_dataset

#Limpiar la memoria de la GPU a la fuerza
torch.cuda.empty_cache()
gc.collect()
print(" Memoria de GPU limpiada.")

#Cargar el dataset limpio desde cero
dataset = load_dataset("mteb/stsbenchmark-sts")
print("Dataset cargado correctamente.")

BLOQUE 2.2: PREPOROCESADO INTELIGENTE

El modelo BERT necesita que las frases estén convertidas en números, y el Trainer exige que la columna de notas se llame Label para poder procesarla. Así, en este bloque tokenizamos frases juntas, renonbramos y detectamos columnas que sobren para borrarlas.


In [ ]:
# BLOQUE Z2 - PASO 2: PREPROCESADO
from transformers import AutoTokenizer

# 1. Cargamos el tokenizador
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 2. Función para preparar los datos
def preprocess_function(examples):
    # Tokenizamos las dos frases juntas (Frase A [SEP] Frase B)
    tokenized = tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, max_length=128)
    tokenized['label'] = [float(score) for score in examples['score']]
    return tokenized

# 3. Aplicamos la transformación
print("Tokenizando datos")
encoded_dataset = dataset.map(preprocess_function, batched=True)

columnas_necesarias = ['input_ids', 'token_type_ids', 'attention_mask', 'label']
columnas_existentes = encoded_dataset["train"].column_names
columnas_a_borrar = [col for col in columnas_existentes if col not in columnas_necesarias]

print(f"Borrado de columnas innecesarias: {columnas_a_borrar}")
encoded_dataset = encoded_dataset.remove_columns(columnas_a_borrar)

# 5. Formato final para PyTorch
encoded_dataset.set_format("torch")
print("Datos listos para el entrenamiento.")
print("Ejemplo de entrada:", encoded_dataset['train'][0].keys())

BLOQUE 3: CONFIGURAMOS EL ENTRENADOR

Voy a definir el modelo, metrica y reglas que vamos a usar para el entrenamiento. Lo voy a hacer con una configuración de tamaño de batch de 8 para asegurar que no se sature la memoria de collab, pero usando a su vez acumulazion de gradiantes que le va a yudar en el aprendizaje.

---



In [ ]:
# BLOQUE Z2 - PASO 3: MODELO Y CONFIGURACIÓN
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from scipy.stats import pearsonr
import numpy as np

# 1. Cargamos el modelo (num_labels=1 para Regresión)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

# 2. Definimos la métrica (Pearson)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Protección por si devuelve una tupla extraña
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    # Aplanamos y calculamos Pearson
    return {"pearson": pearsonr(predictions.flatten(), labels.flatten())[0]}

# 3. Argumentos de Entrenamiento mejores
args = TrainingArguments(
    output_dir="modelo_z2_final",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,              # Variable que va cambiando para el procesamiento de distintas pruebas
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,   # CAMBIO TAMBIEN (EN BATCH DE 16 A 32 PORQUE LA MULTIPLICACION SE HARÁ CON ESTE)
    per_device_eval_batch_size=16,
    num_train_epochs=10,              # los epoches que le damos -cuantas vueltas- (CAMBIA)
    weight_decay=0.01,
    load_best_model_at_end=True,     # Quedarnos con el mejor
    metric_for_best_model="pearson",
    report_to="none",
    fp16=True
)

# 4. Inicializamos el Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print("Entrenador configurado")

2.4: ENTRENAMIENTO

Vamos a entrenar con los parametros que hemos metido antes y los datos que hemos adecuado y vamos a ver como reacciona nuestro sistema con la nota que nos da







In [ ]:
# --- BLOQUE Z2 - PASO 4: ENTRENAMIENTO Y RESULTADO ---

print("Iniciando entrenamiento Z2")
trainer.train()

print("\nEntrenamiento finalizado. Evaluando en el conjunto de TEST...")
# Hacemos la predicción final sobre el examen real (Test)
test_results = trainer.predict(encoded_dataset["test"])
pearson_final = test_results.metrics['test_pearson']

print("="*40)
print(f"RESULTADO FINAL Z2 (BERT Fine-Tuning): {pearson_final:.4f}")
print("="*40)

**CAMBIO DE ARQUITECTURA PARA VER CUAL NOS BENEFICIA MÁS CON RESPECTO A LO QUE TENEMOS HASTA AHORA:**

_CAMBIO 1: OPTIMIZACIÓN DEL BERT INICIAL_

Las correcciones que he aplicado a este bloque son las siguientes: por un lado, le he agregado un early stopping (frente al solo Load_best_model_at_end con el que contaba al principio) y luego he agregado una semilla para que sea reproducible. Además, le he agregado un learnin rate estático, con un warmup de 0.1, que puede ayudarme a progresar más adecuadamente en los experiemntos.

También le he agregado un seed para que cada vez que inicialice con los mismos parámetros mirando a futuras pruebas o entrenamientos que se puedan hacer sobre el código, obteniendo así resultados consistentes.

In [ ]:
# --- BLOQUE 2.5: BERT OPTIMIZADO (Mejoras de Hiperparámetros) ---
from transformers import EarlyStoppingCallback, set_seed

# 1. Fijo semilla para reproducibilidad
set_seed(42)

# 2. Argumentos de Entrenamiento Mejorados
args_optimized = TrainingArguments(
    output_dir="modelo_z2_bert_opt",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,              # Mantengo el LR con el que conseguimos mejores resultados
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,   # Batch de 32 (8x4)
    per_device_eval_batch_size=16,
    num_train_epochs=10,             # Le doy margen de 10 epoches, aunque con el early stopping parará.
    weight_decay=0.01,
    warmup_ratio=0.1,                # WARMUP: suaviza aprendizaje para evitar overfitting
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,
    save_total_limit=2,              # Ahorra espacio guardando solo los 2 mejores
    report_to="none",
    fp16=True
)

# 3. Reinicializar el modelo para entrenarlo de 0
model_opt = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

# 4. Trainer con Early Stopping
trainer_opt = Trainer(
    model=model_opt,
    args=args_optimized,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]) # EARLY STOPPING para que pare si no ve mejoras en 3 épocas

print("Inicio entrenamiento BERT Optimizado")
trainer_opt.train()

# 5. Evaluación Final
print("\n Evaluando BERT Optimizado en TEST")
results_opt = trainer_opt.predict(encoded_dataset["test"])
print(f"RESULTADO FINAL (BERT Optimizado): {results_opt.metrics['test_pearson']:.4f}")

**EXPERIMENTO CON OTRA ARQUITECTURA: RoBERTa**

Prueba con Roberta para optimizar el pre-entrenamiento de BERT (elimina el NSP, usa dynamic masking) y es la base arquitectonica del modelo multilingüe de XLM-ROBERTA, que es el que estaré utilizando para el modelo de Z3.


Roberta es la evolución directa de BERT. Los autores de RoBERTa demostraron que BERT estaba "sub-entrenado". Al quitar la tarea de predicción de la siguiente frase (NSP) y entrenar con más datos, lograron un rendimiento muy superior en tareas semánticas como STS.



In [ ]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from scipy.stats import pearsonr

# 1. LIMPIEZA DE MEMORIA
del model_opt, trainer_opt  # Borro los objetos anteriores si existen
torch.cuda.empty_cache()
gc.collect()
print("Memoria liberada. Preparando experimento RoBERTa")

# 2. CONFIGURACIÓN DEL NUEVO MODELO
model_checkpoint = "roberta-base"
print(f"Cargando {model_checkpoint}")

tokenizer_roberta = AutoTokenizer.from_pretrained(model_checkpoint)
model_roberta = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=1)

# 3. PREPROCESADO ESPECÍFICO
def preprocess_roberta(examples):
    return tokenizer_roberta(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        max_length=128
    )

print("Tokenizando dataset para RoBERTa")
encoded_dataset_roberta = dataset.map(preprocess_roberta, batched=True)

# Convertir score a float y formateamos
encoded_dataset_roberta = encoded_dataset_roberta.map(
    lambda x: {'label': float(x['score'])}
)
# Eliminar columnas que no sean las necesarias
cols_to_keep = ['input_ids', 'attention_mask', 'label']
cols_to_remove = [c for c in encoded_dataset_roberta['train'].column_names if c not in cols_to_keep]
encoded_dataset_roberta = encoded_dataset_roberta.remove_columns(cols_to_remove)
encoded_dataset_roberta.set_format("torch")

# 4. CONFIGURACIÓN DE ENTRENAMIENTO

args_roberta = TrainingArguments(
    output_dir="modelo_z2_roberta_experiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,              # Mantener LR exitoso
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,   # Batch efectivo 32
    per_device_eval_batch_size=16,
    num_train_epochs=10,             # Con Early Stopping
    weight_decay=0.01,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

trainer_roberta = Trainer(
    model=model_roberta,
    args=args_roberta,
    train_dataset=encoded_dataset_roberta["train"],
    eval_dataset=encoded_dataset_roberta["validation"],
    tokenizer=tokenizer_roberta, # Pasar el tokenizer nuevo
    compute_metrics=compute_metrics, # Reusar función de métricas
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 5. EJECUCIÓN
print("Entrenando RoBERTa-base")
trainer_roberta.train()

# 6. EVALUACIÓN COMPARATIVA
print("\n RESULTADOS EXPERIMENTO ROBERTA (TEST SET):")
results_roberta = trainer_roberta.predict(encoded_dataset_roberta["test"])
pearson_roberta = results_roberta.metrics['test_pearson']

print(f"Pearson RoBERTa: {pearson_roberta:.4f}")
print("-" * 30)

**Z3: TRANSFERENCIA ENTRE LENGUAS**

El objetivo de esta útlima parte es ver si es posible aplicar lo aprendido en un idioma (inglés) a otro que no ha sido visto durante el entrenamiento.


_BLOQUE 3.1: CARGA DE DATOS EN ESPAÑOL Y EVALUACION DEL MODELO Z1 Y Z2 (RoBERTAa y BERT) PARA ESTE IDIOMA_


Lo que quiero demostrar con la utilización de estos modelos es que un modelo monolingüe no funciona en español, significando esto que es necesario la utilización de modelos monolingües.

**Carga de datos en Español**

In [ ]:
from datasets import load_dataset
# 1. Cargar el dataset de Test en Español
# Usar 'stsb_multi_mt', que es el estándar para STS multilingüe
print("Cargando dataset de test en Español (stsb_multi_mt)...")
try:
    dataset_es = load_dataset("stsb_multi_mt", name="es", split="test")
    print("Dataset stsb_multi_mt cargado correctamente.")
except Exception as e:
    print(f"Error cargando stsb_multi_mt: {e}")
    print("Intentando cargar desde mteb/stsbenchmark-sts (configuración 'es')...")
    dataset_es = load_dataset("mteb/stsbenchmark-sts", "es", split="test")


1. PRUEBA DE LOS DATOS MULTILINGÜES CON LOS ENFOQUES NO SUPERVISADOS:
  * N-Gramas: Para ver si las frases se parecen solo por compartir palabras.

* Sentence Embeddings: Usaré el modelo distiluse-base-multilingual-cased-v2 como referencia.


In [ ]:

# BLOQUE 3.1b: EVALUACIÓN DE ENFOQUES NO SUPERVISADOS (Z1) EN ESPAÑOL
# Objetivo: Evaluar N-Gramas y el modelo Z1 original sobre el test en Español.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from scipy.stats import pearsonr
import numpy as np
from datasets import load_dataset

test_sentences1_es = list(dataset_es['sentence1'])
test_sentences2_es = list(dataset_es['sentence2'])


label_col = 'similarity_score' if 'similarity_score' in dataset_es.features else 'score'
gold_scores_es = list(dataset_es[label_col])

print(f"Datos listos: {len(test_sentences1_es)} pares de frases.")

# --- EXPERIMENTO A: N-GRAMAS (Español) ---
print("1. Ejecutando N-Gramas en Español")

vectorizer_es = CountVectorizer().fit(test_sentences1_es + test_sentences2_es)
vecs1_es = vectorizer_es.transform(test_sentences1_es)
vecs2_es = vectorizer_es.transform(test_sentences2_es)

ngram_preds_es = []
for i in range(len(test_sentences1_es)):
    # Similitud coseno
    sim = cosine_similarity(vecs1_es[i], vecs2_es[i])[0][0]
    ngram_preds_es.append(sim * 5)

pearson_ngram_es = pearsonr(ngram_preds_es, gold_scores_es)[0]

# --- EXPERIMENTO B: SENTENCE EMBEDDINGS Z1 ---
z1_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
print(f"2. Ejecutando modelo Z1 ({z1_model_name}) en Español")

model_st_z1 = SentenceTransformer(z1_model_name)

# Codificamos
embeddings1_es = model_st_z1.encode(test_sentences1_es, convert_to_tensor=True)
embeddings2_es = model_st_z1.encode(test_sentences2_es, convert_to_tensor=True)

# Calculamos similitud
cosine_scores_es = util.cos_sim(embeddings1_es, embeddings2_es)

st_preds_es = []
for i in range(len(test_sentences1_es)):
    st_preds_es.append(cosine_scores_es[i][i].item() * 5)

pearson_st_es = pearsonr(st_preds_es, gold_scores_es)[0]

# --- RESULTADOS ---
print("\n" + "="*50)
print(f"RESULTADOS BASALES EN ESPAÑOL (Subconjunto Z1):")
print(f"1. N-Gramas (Coincidencia léxica): {pearson_ngram_es:.4f}")
print(f"2. Embeddings Z1 (all-MiniLM - Inglés): {pearson_st_es:.4f}")
print("="*50)

2. EVALUACIÓN CON ROBERTA (Z2)

In [ ]:

import numpy as np
# 2. Preprocesar datos en Español para RoBERTa
# IMPORTANTE: Uso el tokenizador de RoBERTa (que solo sabe inglés) para ver cómo falla al intentar entender español.
def preprocess_es_roberta(examples):
    tokenized = tokenizer_roberta(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        max_length=128
    )

    label_col = 'similarity_score' if 'similarity_score' in examples else 'score'
    tokenized['label'] = [float(score) for score in examples[label_col]]
    return tokenized

print("Tokenizando español con tokenizador de RoBERTa en inglés")
encoded_es_roberta = dataset_es.map(preprocess_es_roberta, batched=True)
encoded_es_roberta.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])

# 3. Evaluación Zero-Shot (Control Negativo)
print("\nEvaluando RoBERTa (Inglés) en datos Español")
results_control = trainer_roberta.predict(encoded_es_roberta)
pearson_control = results_control.metrics['test_pearson']

print("="*50)
print(f"RESULTADO CONTROL (RoBERTa Inglés en Español): {pearson_control:.4f}")
print("="*50)

Al ver que me da bastante alto, le he metido prints para ver que era lo que esta utlizando al codificar, puesto que a veces se realiza una traduccion y podría ser por eso que diese los datos tan altos.

In [ ]:
# VERIFICACIÓN DE DATOS
print("--- Muestra de datos de Test ---")
print(f"Frase 1 (Original): {dataset_es[0]['sentence1']}")
print(f"Frase 2 (Original): {dataset_es[0]['sentence2']}")
print(f"Score: {dataset_es[0]['similarity_score'] if 'similarity_score' in dataset_es[0] else dataset_es[0]['score']}")

# Ver qué tokens ve el modelo
print("\n--- Lo que ve el Tokenizador ---")
input_ids_ejemplo = encoded_es_roberta[0]['input_ids']
print(f"Tokens decodificados: {tokenizer_roberta.decode(input_ids_ejemplo)}")

3. EVALUACION DEL MODELO Z2 (BERT) PARA ESTE IDIOMA:

Como he limpiado antes de seguir con este bloque nos va a dar error asi que hay que volver a ejecutar el Z2 de BERT para volver a conseguir los datos que necesitamos para proceder con al ejecución de BERT para español


In [ ]:
#Carga del dataset en español
if 'dataset_es' not in globals():
    from datasets import load_dataset
    print("Cargando dataset en Español...")
    try:
        dataset_es = load_dataset("stsb_multi_mt", name="es", split="test")
    except:
        dataset_es = load_dataset("mteb/stsbenchmark-sts", "es", split="test")

#Preprocesar Español con el tokenizador de BERT
def preprocess_es_bert_trained(examples):
    tokenized = tokenizer(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        max_length=128
    )

    label_key = 'similarity_score' if 'similarity_score' in examples else 'score'
    tokenized['label'] = [float(s) for s in examples[label_key]]
    return tokenized

print("Tokenizando español con el tokenizador de BERT")
encoded_es_bert_tuned = dataset_es.map(preprocess_es_bert_trained, batched=True)
encoded_es_bert_tuned.set_format("torch", columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

print("\n🇪🇸 Evaluando tu BERT Optimizado (Z2) en datos Españo")
results_bert_tuned = trainer_opt.predict(encoded_es_bert_tuned)
pearson_bert_tuned = results_bert_tuned.metrics['test_pearson']

print("="*50)
print(f"RESULTADO BERT OPTIMIZADO (Z2) EN ESPAÑOL: {pearson_bert_tuned:.4f}")
print("="*50)

_3.2 ENTRENAMIENTO DEL MODELO MULTILINGÜE: FINE-TUNING CROSS-LINGUAL CON XLM-ROBERTA:_

In [ ]:
import torch
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    set_seed
)
from datasets import load_dataset
from scipy.stats import pearsonr
import numpy as np

# 1. LIMPIEZA DE MEMORIA
# Borrar variables de modelos anteriores si existen para liberar la GPU
for var in ['model_opt', 'trainer_opt', 'model_roberta', 'trainer_roberta', 'model_bert_audit', 'trainer_xlm']:
    if var in globals(): del globals()[var]

torch.cuda.empty_cache()
gc.collect()
set_seed(42)
print("Memoria liberada")

# 2. CARGAR ARQUITECTURA
model_checkpoint = "xlm-roberta-base"

tokenizer_xlm = AutoTokenizer.from_pretrained(model_checkpoint)
model_xlm = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=1)

# 3. PREPARACIÓN DE DATOS DE ENTRENAMIENTO (INGLÉS)
def preprocess_xlm(examples):
    # El dataset inglés usa 'score'
    label_col = 'score' if 'score' in examples else 'similarity_score'
    tokenized = tokenizer_xlm(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        max_length=128
    )
    tokenized['label'] = [float(s) for s in examples[label_col]]
    return tokenized

print("Cargando datos de ENTRENAMIENTO (Inglés)")
dataset_en = load_dataset("mteb/stsbenchmark-sts")

print("Tokenizando datos de entrenamiento...")
encoded_train_en = dataset_en.map(preprocess_xlm, batched=True)

# Formato Torch para el entrenamiento
cols = ['input_ids', 'attention_mask', 'label']
for split in ['train', 'validation']:
    encoded_train_en[split].set_format("torch", columns=cols)

# 4. CONFIGURACIÓN DEL ENTRENAMIENTO
args_xlm = TrainingArguments(
    output_dir="modelo_z3_xlm_roberta",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,              # XLM-R suele preferir LRs conservadores
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,   # Batch efectivo = 32
    per_device_eval_batch_size=16,
    num_train_epochs=10,             # Con margen para el Early Stopping
    weight_decay=0.01,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple): predictions = predictions[0]
    return {"pearson": pearsonr(predictions.flatten(), labels.flatten())[0]}

trainer_xlm = Trainer(
    model=model_xlm,
    args=args_xlm,
    train_dataset=encoded_train_en["train"],
    eval_dataset=encoded_train_en["validation"],
    tokenizer=tokenizer_xlm,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 5. EJECUTAR ENTRENAMIENTO
print("Iniciando entrenamiento")
trainer_xlm.train()

print("Entrenamiento finalizado")

_3.3 EVALUACIÓN FINAL Y CAMBIOS A REALIZAR PARA EL ENTRENAMIENTO_

Utilizando los parametros iniciales del entrenamiento, los que me funcionaron bien en el entrenamiento anterior.

In [ ]:

# BLOQUE 3.3: EVALUACIÓN FINAL Z3 (Test en Español)

# 1. CARGAR Y PREPARAR DATOS DE TEST (ESPAÑOL)
print("Preparando datos de TEST (Español)")
if 'dataset_es' not in globals():
    from datasets import load_dataset
    try:
        dataset_es = load_dataset("stsb_multi_mt", name="es", split="test")
    except:
        dataset_es = load_dataset("mteb/stsbenchmark-sts", "es", split="test")

# Tokenizador de memoria
def preprocess_xlm_test(examples):
    label_col = 'score' if 'score' in examples else 'similarity_score'
    tokenized = tokenizer_xlm(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        max_length=128
    )
    tokenized['label'] = [float(s) for s in examples[label_col]]
    return tokenized

encoded_test_es = dataset_es.map(preprocess_xlm_test, batched=True)
encoded_test_es.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])

# 2. EVALUACIÓN
print("\n Evaluando modelo XLM-R Base en ESPAÑOL")
# Uso el 'trainer_xlm' que ya tiene el mdelo cargado
results_xlm = trainer_xlm.predict(encoded_test_es)
pearson_xlm = results_xlm.metrics['test_pearson']

print("="*50)
print(f"RESULTADO FINAL Z3 (XLM-R Base): {pearson_xlm:.4f}")
print("="*50)

**EXPERIMENTOS PARA ESTABLECER LOS MEJORES PARÁMETROS**

In [ ]:

# BLOQUE 0: CARGA DE DATOS COMÚN (Ejecutar UNA vez)

import torch
import gc
from transformers import AutoTokenizer, set_seed
from datasets import load_dataset

# 1. Configuración Global
model_checkpoint = "xlm-roberta-base"
tokenizer_xlm = AutoTokenizer.from_pretrained(model_checkpoint)

# 2. Cargar Datos (si no están ya)
print("Cargando datos comunes")
dataset_en = load_dataset("mteb/stsbenchmark-sts")
try:
    dataset_es = load_dataset("stsb_multi_mt", name="es", split="test")
except:
    dataset_es = load_dataset("mteb/stsbenchmark-sts", "es", split="test")

# 3. Tokenizar todo de una vez
def preprocess_xlm(examples):
    label_col = 'score' if 'score' in examples else 'similarity_score'
    tokenized = tokenizer_xlm(
        examples['sentence1'],
        examples['sentence2'],
        truncation=True,
        max_length=128
    )
    tokenized['label'] = [float(s) for s in examples[label_col]]
    return tokenized

print(" Tokenizando")
encoded_train = dataset_en.map(preprocess_xlm, batched=True)
encoded_test_es = dataset_es.map(preprocess_xlm, batched=True)

# Formato Torch
cols = ['input_ids', 'attention_mask', 'label']
for split in ['train', 'validation']:
    encoded_train[split].set_format("torch", columns=cols)
encoded_test_es.set_format("torch", columns=cols)

print("Carga completa")

In [ ]:
# EXP 1: BASELINE (Referencia)
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from scipy.stats import pearsonr

# Limpieza
torch.cuda.empty_cache(); gc.collect(); set_seed(42)

print("\n INICIANDO EXP 1: BASELINE")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

args = TrainingArguments(
    output_dir="z3_exp1_base",
    eval_strategy="epoch", save_strategy="epoch",
    learning_rate=2e-5,              # LR Estándar
    per_device_train_batch_size=8, gradient_accumulation_steps=4,
    num_train_epochs=10,
    weight_decay=0.01, warmup_ratio=0.1,
    load_best_model_at_end=True, metric_for_best_model="pearson", greater_is_better=True,
    save_total_limit=1, fp16=True, report_to="none"
)

def compute_metrics(p): return {"pearson": pearsonr(p.predictions.flatten(), p.label_ids.flatten())[0]}

trainer = Trainer(
    model=model, args=args,
    train_dataset=encoded_train["train"], eval_dataset=encoded_train["validation"],
    tokenizer=tokenizer_xlm, compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
res = trainer.predict(encoded_test_es)
print(f" RESULTADO EXP 1 (Baseline): {res.metrics['test_pearson']:.4f}")

In [ ]:
# --- EXP 2: LR ALTO ---
torch.cuda.empty_cache(); gc.collect(); set_seed(42)

print("\n INICIANDO EXP 2: HIGH LEARNING RATE")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

args = TrainingArguments(
    output_dir="z3_exp2_high_lr",
    eval_strategy="epoch", save_strategy="epoch",
    learning_rate=5e-5,              # CAMBIO: 2.5x más rápido
    per_device_train_batch_size=8, gradient_accumulation_steps=4,
    num_train_epochs=10,
    weight_decay=0.01, warmup_ratio=0.1,
    load_best_model_at_end=True, metric_for_best_model="pearson", greater_is_better=True,
    save_total_limit=1, fp16=True, report_to="none"
)

trainer = Trainer(
    model=model, args=args,
    train_dataset=encoded_train["train"], eval_dataset=encoded_train["validation"],
    tokenizer=tokenizer_xlm, compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
res = trainer.predict(encoded_test_es)
print(f" RESULTADO EXP 2 (High LR 5e-5): {res.metrics['test_pearson']:.4f}")

In [ ]:
# --- EXP 3: LR BAJO  ---
torch.cuda.empty_cache(); gc.collect(); set_seed(42)

print("\n INICIANDO EXP 3: LOW LEARNING RATE")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

args = TrainingArguments(
    output_dir="z3_exp3_low_lr",
    eval_strategy="epoch", save_strategy="epoch",
    learning_rate=1e-5,              # CAMBIO Mitad de velocidad
    per_device_train_batch_size=8, gradient_accumulation_steps=4,
    num_train_epochs=12,             # CAMBI Un poco más de tiempo para compensar
    weight_decay=0.01, warmup_ratio=0.1,
    load_best_model_at_end=True, metric_for_best_model="pearson", greater_is_better=True,
    save_total_limit=1, fp16=True, report_to="none"
)

trainer = Trainer(
    model=model, args=args,
    train_dataset=encoded_train["train"], eval_dataset=encoded_train["validation"],
    tokenizer=tokenizer_xlm, compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
res = trainer.predict(encoded_test_es)
print(f" RESULTADO EXP 3 (Low LR 1e-5): {res.metrics['test_pearson']:.4f}")

In [ ]:
# --- EXPERIMENTO 4: HIGH LEARNING RATE (5e-5) ---
# Hipótesis: Un learning rate más alto podría permitir al modelo adaptarse
# más rápido a la tarea, aunque corre riesgo de inestabilidad.

import torch
import gc
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, set_seed
from scipy.stats import pearsonr

# 1. Limpieza
if 'model' in globals(): del model
if 'trainer' in globals(): del trainer
torch.cuda.empty_cache(); gc.collect(); set_seed(42)

# 2. Modelo Nuevo
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

# 3. Configuración
args_exp4 = TrainingArguments(
    output_dir="z3_exp4_high_lr",
    eval_strategy="epoch", save_strategy="epoch",

    learning_rate=5e-5,              # CAMBIO Más alto (antes era 2e-5)

    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,   # Batch efectivo = 32
    num_train_epochs=10,
    weight_decay=0.01, warmup_ratio=0.1,
    load_best_model_at_end=True, metric_for_best_model="pearson", greater_is_better=True,
    save_total_limit=1, fp16=True, report_to="none"
)
#DEFINIR METRICA
def compute_metrics(p): return {"pearson": pearsonr(p.predictions.flatten(), p.label_ids.flatten())[0]}

trainer = Trainer(
    model=model, args=args_exp4,
    train_dataset=encoded_train["train"], eval_dataset=encoded_train["validation"],
    tokenizer=tokenizer_xlm, compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
res = trainer.predict(encoded_test_es)
print(f" RESULTADO EXP 4 (High LR): {res.metrics['test_pearson']:.4f}")

In [ ]:
# --- EXPERIMENTO 5: SMALL BATCH SIZE (Sin Acumulación) ---
# Hipótesis: Al quitar la acumulación de gradientes, el modelo actualiza sus pesos
# con cada batch de 8 ejemplos (más ruido). Quiero ver si esto ayuda o perjudica.

# 1. Limpieza
torch.cuda.empty_cache(); gc.collect(); set_seed(42)

print("\n INICIANDO EXP 5: BATCH SIZE PEQUEÑO (8)")

# 2. Modelo Nuevo
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

# 3. Configuración
args_exp5 = TrainingArguments(
    output_dir="z3_exp5_small_batch",
    eval_strategy="epoch", save_strategy="epoch",
    learning_rate=2e-5,              # LR normal

    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,   # CAMBIO: Antes 4 (Batch 32), ahora 1 (Batch 8)
                                     # Esto hace el entrenamiento más "inestable" pero a veces explora mejor.

    num_train_epochs=10,
    weight_decay=0.01, warmup_ratio=0.1,
    load_best_model_at_end=True, metric_for_best_model="pearson", greater_is_better=True,
    save_total_limit=1, fp16=True, report_to="none"
)

trainer = Trainer(
    model=model, args=args_exp5,
    train_dataset=encoded_train["train"], eval_dataset=encoded_train["validation"],
    tokenizer=tokenizer_xlm, compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
res = trainer.predict(encoded_test_es)
print(f" RESULTADO EXP 5 (Small Batch): {res.metrics['test_pearson']:.4f}")

In [ ]:
# --- EXP 6: HIGH STABILITY ---
torch.cuda.empty_cache(); gc.collect(); set_seed(42)

print("\n INICIANDO EXP 6")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

args = TrainingArguments(
    output_dir="z3_exp6_stable",
    eval_strategy="epoch", save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,   # CAMBIO: Batch Efectivo = 64
    num_train_epochs=20,             # CAMBIO: Hasta 20 epoches
    weight_decay=0.01, warmup_ratio=0.1,
    load_best_model_at_end=True, metric_for_best_model="pearson", greater_is_better=True,
    save_total_limit=1, fp16=True, report_to="none"
)

trainer = Trainer(
    model=model, args=args,
    train_dataset=encoded_train["train"], eval_dataset=encoded_train["validation"],
    tokenizer=tokenizer_xlm, compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)] # CAMBIO EN EL EAERLY STOPPING
)

trainer.train()
res = trainer.predict(encoded_test_es)
print(f"RESULTADO EXP 6 (High Stability): {res.metrics['test_pearson']:.4f}")

**ULTIMOS EXPERIEMNTOS: VER LA TRANSFERENCIA ENTRE DISTINTOS IDIOMAS QUE PUEDAN TENER DISNTINTA FORMA DEL ESPAÑOL Y VER COMO REACCIONAN - TESTEAR TRASNGFERENCIA**

_BLOQUE 1: Preparación para los últimos experimentos_

In [ ]:
# --- BLOQUE 0: PREPARACIÓN ---
from datasets import concatenate_datasets, load_dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AutoTokenizer
from scipy.stats import pearsonr
import numpy as np

# 1. Cargar Tokenizer
tokenizer_xlm = AutoTokenizer.from_pretrained("xlm-roberta-base")

# 2. Funciones Auxiliares
def get_multilingual_data(languages, split):
    """Descarga y combina idiomas desde stsb_multi_mt"""
    datasets_list = []
    print(f"Cargando datos para: {languages} (split: {split})")
    for lang in languages:
        try:
            ds = load_dataset("mteb/stsb_multi_mt", name=lang, split=split)
            if 'similarity_score' in ds.features:
                 ds = ds.rename_column("similarity_score", "label")
            datasets_list.append(ds)
        except Exception as e:
            print(f"Error cargando idioma {lang}: {e}")
    return concatenate_datasets(datasets_list) if datasets_list else None

def preprocess_function(examples):
    return tokenizer_xlm(examples['sentence1'], examples['sentence2'], truncation=True, max_length=128)

def compute_metrics(p):
    return {"pearson": pearsonr(p.predictions.flatten(), p.label_ids.flatten())[0]}

# 3. Configuración de entrenamiento manteniendo mejores hiperparámetros
common_args = TrainingArguments(
    output_dir="checkpoints_temp",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)
print("Configuración lista.")

_PRIMER EXPERIMENTO: ENTRENAMIENTO EN INGLÉS Y CASTELLANO Y POSTERIOR TEST EN RUSO_

(generalización en distinto alfabéto: cirílico)

In [ ]:
# EXPERIMENTO 1: TRAIN [ES + EN] -> TEST [RU]
print("\n INICIANDO EXP 1: Políglota (ES+EN) probando en RUSO")

# 1. Datos
train_ds = get_multilingual_data(['es', 'en'], 'train').map(preprocess_function, batched=True)
val_ds   = get_multilingual_data(['es', 'en'], 'dev').map(preprocess_function, batched=True)
test_ds  = get_multilingual_data(['ru'], 'test').map(preprocess_function, batched=True)

# 2. Inicializar modelo pre entrebado
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

# 3. Entrenar
trainer = Trainer(
    model=model,
    args=common_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer_xlm,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)
trainer.train()

# 4. Evaluar
res = trainer.predict(test_ds)
print(f"\nRESULTADO FINAL EXP 1 (Test RUSO): {res.metrics['test_pearson']:.4f}")

_EXPERIMENTO 2: TRAIN EN ESPAÑOL Y TRANSFERENCIA A CHINO_

De alfabeto latino a logogramas chinos.

In [ ]:
# --- EXPERIMENTO 2: TRAIN [ES] -> TEST [ZH] (Chino) ---
print("\n INICIANDO EXP 2: Español puro probando en CHINO")

# 1. Datos
train_ds = get_multilingual_data(['es'], 'train').map(preprocess_function, batched=True)
val_ds   = get_multilingual_data(['es'], 'dev').map(preprocess_function, batched=True)
test_ds  = get_multilingual_data(['zh'], 'test').map(preprocess_function, batched=True)

# 2. Modelo Limpio para no usar lo aprendido anteriormente
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

# 3. Entrenar
trainer = Trainer(
    model=model,
    args=common_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer_xlm,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)
trainer.train()

# 4. Evaluar
res = trainer.predict(test_ds)
print(f"\n RESULTADO FINAL EXP 2 (Test CHINO): {res.metrics['test_pearson']:.4f}")

_EXPERIMENTO 3: VUELTA CON RESPECTO A LOS EXPERIEMENTOS INICIALES -testear como se mantiene la tranferencia invirtiendo español e inglés_

Transferencia inversa: vamos a entrenar en español y probar en inglés

In [ ]:
# EXPERIMENTO 3: TRAIN [ES] -> TEST [EN]
print("\n INICIANDO EXP 3: Español puro probando en INGLÉS")

# 1. Datos
train_ds = get_multilingual_data(['es'], 'train').map(preprocess_function, batched=True)
val_ds   = get_multilingual_data(['es'], 'dev').map(preprocess_function, batched=True)
test_ds  = get_multilingual_data(['en'], 'test').map(preprocess_function, batched=True)

# 2. Modelo Limpio
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

# 3. Entrenar
trainer = Trainer(
    model=model,
    args=common_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer_xlm,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)
trainer.train()

# 4. Evaluar
res = trainer.predict(test_ds)
print(f"\n RESULTADO FINAL EXP 3 (Test INGLÉS): {res.metrics['test_pearson']:.4f}")